# PRELUCRAREA SI ANALIZA IMAGINILOR COLOR- Proiect

## Agloritmul:A neighborhood evaluated adaptive vector filter for suppression of impulse noise in color images

### Citirea si schibarea rezolutiei pozelor:

In [1]:
from skimage import io,color,util
import matplotlib.pyplot as plt
import numpy as np
from skimage.transform import rescale, resize
import skimage as sk


size = (120, 160)
figs=(40,60)
fig=plt.figure(figsize=figs)
img1 = io.imread('img/pic1.jpg')
img1 = resize(img1, size).astype(np.float32)*255

img2 = io.imread('img/pic2.jpg')
img2 = resize(img2, size).astype(np.float32)*255

img3 = io.imread('img/pic3.jpg')
img3 = resize(img3, size).astype(np.float32)*255

img4 = io.imread('img/pic4.jpg')
img4 = resize(img4, size).astype(np.float32)*255

img5 = io.imread('img/pic5.jpg')
img5 = resize(img5, size).astype(np.float32)*255

img6 = io.imread('img/pic6.jpg')
img6 = resize(img6, size).astype(np.float32)*255
print(np.max(img1))

fig.suptitle('Imagini initiale', fontsize=20)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea 1')
ax2.title.set_text('Imaginea 2')
ax3.title.set_text('Imaginea 3')
ax4.title.set_text('Imaginea 4')
ax5.title.set_text('Imaginea 5')
ax6.title.set_text('Imaginea 6')
ax1.imshow(img1.astype(np.uint8))
ax2.imshow(img2.astype(np.uint8))
ax3.imshow(img3.astype(np.uint8))
ax4.imshow(img4.astype(np.uint8))
ax5.imshow(img5.astype(np.uint8))
ax6.imshow(img6.astype(np.uint8))
plt.show()

C:\Users\Nitroi\AppData\Local\Continuum\anaconda3\envs\PAIC\lib\site-packages\skimage\transform\_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
C:\Users\Nitroi\AppData\Local\Continuum\anaconda3\envs\PAIC\lib\site-packages\skimage\transform\_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


255.0


<Figure size 4000x6000 with 6 Axes>

### Functiile pentru zgomot Gaussian, respectiv zgomot impulsiv.

In [2]:
import os
import cv2
def gaussianNoise(image,sigma):
      row,col,ch= image.shape
      mean = 0 
      gauss = np.random.normal(mean,sigma,(row,col,ch))
      gauss = gauss.reshape(row,col,ch)
      noisy = image + gauss
      return noisy

def saltpepper(image,area):
      row,col,ch = image.shape
      s_vs_p = 0.5
      amount = area/2
      #half for salt and half for pepper
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out


### Adaugarea zgomotului pe imagini

In [3]:
Lista_imgZgomotGaussian=[]
Lista_imgZgomotImpulsiv=[]

img1ZG=gaussianNoise(img1,0.1)
img1ZI=saltpepper(img1,0.1)
Lista_imgZgomotGaussian.append(img1ZG)
Lista_imgZgomotImpulsiv.append(img1ZI)

img2ZG=gaussianNoise(img2,0.1)
img2ZI=saltpepper(img2,0.1)
Lista_imgZgomotGaussian.append(img2ZG)
Lista_imgZgomotImpulsiv.append(img2ZI)

img3ZG=gaussianNoise(img3,0.1)
img3ZI=saltpepper(img3,0.1)
Lista_imgZgomotGaussian.append(img3ZG)
Lista_imgZgomotImpulsiv.append(img3ZI)

img4ZG=gaussianNoise(img4,0.1)
img4ZI=saltpepper(img4,0.1)
Lista_imgZgomotGaussian.append(img4ZG)
Lista_imgZgomotImpulsiv.append(img4ZI)

img5ZG=gaussianNoise(img5,0.1)
img5ZI=saltpepper(img5,0.1)
Lista_imgZgomotGaussian.append(img5ZG)
Lista_imgZgomotImpulsiv.append(img5ZI)

img6ZG=gaussianNoise(img6,0.1)
img6ZI=saltpepper(img6,0.1)
Lista_imgZgomotGaussian.append(img6ZG)
Lista_imgZgomotImpulsiv.append(img6ZI)

C:\Users\Nitroi\AppData\Local\Continuum\anaconda3\envs\PAIC\lib\site-packages\ipykernel_launcher.py:21: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\Users\Nitroi\AppData\Local\Continuum\anaconda3\envs\PAIC\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


### Implementarea filtrelor clasice mediere si medie aritmetca

In [4]:
def medianFilter(img):
    L,l,_=img.shape    
    imag = img.astype(np.float64).copy()
    for i in range(1,L-1):
        for j in range(1,l-1):
            vR=np.sort(img[i-1:i+2,j-1:j+2,0],axis=None)
            vG=np.sort(img[i-1:i+2,j-1:j+2,1],axis=None)
            vB=np.sort(img[i-1:i+2,j-1:j+2,2],axis=None)
            mean=np.array([vR[4],vG[4],vB[4]])
            imag [i,j,0] = vR[4]
            imag [i,j,1] = vG[4]
            imag [i,j,2] = vB[4]
    return imag
def aritmeticFilter(img):
    L,l,_=img.shape
    imag = img.astype(np.float64).copy()
    for i in range(1,L-1):
        for j in range(1,l-1):
            imag [i,j,0] = np.sum(img[i-1:i+2,j-1:j+2,0])/9
            imag [i,j,1] = np.sum(img[i-1:i+2,j-1:j+2,1])/9
            imag [i,j,2] = np.sum(img[i-1:i+2,j-1:j+2,2])/9
    return imag

### Implementarea Neighborhood evaluated adaptive filter (NEAVF)

In [23]:
import operator
def AVD(v):
    R=np.zeros(9)
    for i in range(0,9):
        for j in range(1,9):
            R[i]+=np.sqrt((v[i][0]-v[j][0])**2+(v[i][1]-v[j][1])**2+(v[i][2]-v[j][2])**2)
    return R
def DALS(V,index):
    F=((1/4)*(V[4][1]-V[0][1])+5)/((1/index)*(V[index][1]-V[0][1])+1)
    return F
            
def NEAVF(img,h):
    L,l,_=img.shape
    imag = img.astype(np.float64).copy()
    FR=np.ones((L,l))
    FG=np.ones((L,l))
    FB=np.ones((L,l))
    for i in range(1,L-1):
        for j in range(1,l-1):
            v=img[i-1:i+2,j-1:j+2,:].reshape((9,3))            
            v[0],v[4] = v[4],v[0]            
            cV=v[0]
            V=dict(zip(AVD(v),v))
            print(V)        
            #sortare dupa AVD
            V = sorted(V.items(), key=operator.itemgetter(0))
            print(V)
            index = [i for i, tupl in enumerate(V) if tupl[1] == cV][0]        
            #Lambda=7.5, delta=4.5
            if R>=4:
                F[i,j]=DALS(V,index)
            else:
                F[i,j]=1
            #print(FR[i,j])    
                
#     for i in range(1,L-1):
#         for j in range(1,l-1):
#             HR=np.sum(np.power(FR[i-1:i+2,j-1:j+2].reshape(9), 2))
#             HG=np.sum(np.power(FG[i-1:i+2,j-1:j+2].reshape(9), 2))
#             HB=np.sum(np.power(FR[i-1:i+2,j-1:j+2].reshape(9), 2))
#             print(HR)
    return imag

In [24]:
#TEST
plt.figure(figsize=(10,10)), plt.imshow(NEAVF(img1ZI,0.4))

{357.0885234358761: array([ 78., 114.,  67.], dtype=float32), 361.3194229605682: array([ 79.75, 107.75,  77.25], dtype=float32), 448.2886039831234: array([107.25, 134.25,  81.25], dtype=float32), 494.6965229376252: array([43.75, 74.75, 57.75], dtype=float32), 468.2237934294583: array([109.75, 137.75,  79.25], dtype=float32), 857.0765662222616: array([50.25,  1.  , 59.75], dtype=float32), 655.3358089468821: array([  1.  , 116.25,  64.75], dtype=float32), 381.7778370170446: array([ 92.25, 125.75,  69.  ], dtype=float32)}
[(357.0885234358761, array([ 78., 114.,  67.], dtype=float32)), (361.3194229605682, array([ 79.75, 107.75,  77.25], dtype=float32)), (381.7778370170446, array([ 92.25, 125.75,  69.  ], dtype=float32)), (448.2886039831234, array([107.25, 134.25,  81.25], dtype=float32)), (468.2237934294583, array([109.75, 137.75,  79.25], dtype=float32)), (494.6965229376252, array([43.75, 74.75, 57.75], dtype=float32)), (655.3358089468821, array([  1.  , 116.25,  64.75], dtype=float32)), 

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

<Figure size 720x720 with 0 Axes>

## Functiile utilizate la calculul zgomotului

In [ ]:
def PSNR(imgInit, imgFinal):
    mse = np.mean( (imgInit - imgFinal) ** 2 )
    if mse == 0:
        return 100
    PIXEL_MAX = 1
    return 20 * math.log10(PIXEL_MAX / math.sqrt(mse))

def MAE(imgInit, imgFinal):  
    mae = np.sum(np.absolute((imgInit.astype("float") - imgFinal.astype("float"))))
    return mae

## Comparatia rezultatlor celor 3 filtre
### Imaginea 1

In [ ]:
index=0
fig=plt.figure(figsize=figs)
imgZG_afterMedian=medianFilter(Lista_imgZgomotGaussian[index])
imgZG_afterAritmetic=aritmeticFilter(Lista_imgZgomotGaussian[index])


imgZI_afterMedian=medianFilter(Lista_imgZgomotImpulsiv[index])
imgZI_afterAritmetic=aritmeticFilter(Lista_imgZgomotImpulsiv[index])



fig.suptitle('Pentru imaginea 1:', fontsize=50)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea afectata de zgomot gaussian')
ax2.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul median PSNR: '+ str(PSNR(img1,imgZG_afterMedian))+' si MAE: ' + str(MAE(img1,imgZG_afterMedian)))
ax3.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img1,imgZG_afterAritmetic))+' si MAE: ' + str(MAE(img1,imgZG_afterAritmetic)))
ax4.title.set_text('Imaginea afectata de zgomot impulsiv')
ax5.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul median PSNR: '+ str(PSNR(img1,imgZI_afterMedian))+' si MAE: ' + str(MAE(img1,imgZI_afterMedian)))
ax6.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img1,imgZI_afterAritmetic))+' si MAE: ' + str(MAE(img1,imgZI_afterAritmetic)))
ax1.imshow(Lista_imgZgomotGaussian[index])
ax2.imshow(imgZG_afterMedian)
ax3.imshow(imgZG_afterAritmetic)
ax4.imshow(Lista_imgZgomotImpulsiv[index])
ax5.imshow(imgZI_afterMedian)
ax6.imshow(imgZI_afterAritmetic)
plt.show()

### Imaginea 2

In [ ]:
index=1
fig=plt.figure(figsize=figs)
imgZG_afterMedian=medianFilter(Lista_imgZgomotGaussian[index])
imgZG_afterAritmetic=aritmeticFilter(Lista_imgZgomotGaussian[index])


imgZI_afterMedian=medianFilter(Lista_imgZgomotImpulsiv[index])
imgZI_afterAritmetic=aritmeticFilter(Lista_imgZgomotImpulsiv[index])



fig.suptitle('Pentru imaginea 2:', fontsize=50)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea afectata de zgomot gaussian')
ax2.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul median PSNR: '+ str(PSNR(img2,imgZG_afterMedian))+' si MAE: ' + str(MAE(img2,imgZG_afterMedian)))
ax3.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img2,imgZG_afterAritmetic))+' si MAE: ' + str(MAE(img2,imgZG_afterAritmetic)))
ax4.title.set_text('Imaginea afectata de zgomot impulsiv')
ax5.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul median PSNR: '+ str(PSNR(img2,imgZI_afterMedian))+' si MAE: ' + str(MAE(img2,imgZI_afterMedian)))
ax6.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img2,imgZI_afterAritmetic))+' si MAE: ' + str(MAE(img2,imgZI_afterAritmetic)))
ax1.imshow(Lista_imgZgomotGaussian[index])
ax2.imshow(imgZG_afterMedian)
ax3.imshow(imgZG_afterAritmetic)
ax4.imshow(Lista_imgZgomotImpulsiv[index])
ax5.imshow(imgZI_afterMedian)
ax6.imshow(imgZI_afterAritmetic)
plt.show()

### Imaginea 3

In [ ]:
index=2
fig=plt.figure(figsize=figs)
imgZG_afterMedian=medianFilter(Lista_imgZgomotGaussian[index])
imgZG_afterAritmetic=aritmeticFilter(Lista_imgZgomotGaussian[index])


imgZI_afterMedian=medianFilter(Lista_imgZgomotImpulsiv[index])
imgZI_afterAritmetic=aritmeticFilter(Lista_imgZgomotImpulsiv[index])



fig.suptitle('Pentru imaginea 3:', fontsize=50)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea afectata de zgomot gaussian')
ax2.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul median PSNR: '+ str(PSNR(img3,imgZG_afterMedian))+' si MAE: ' + str(MAE(img3,imgZG_afterMedian)))
ax3.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img3,imgZG_afterAritmetic))+' si MAE: ' + str(MAE(img3,imgZG_afterAritmetic)))
ax4.title.set_text('Imaginea afectata de zgomot impulsiv')
ax5.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul median PSNR: '+ str(PSNR(img3,imgZI_afterMedian))+' si MAE: ' + str(MAE(img3,imgZI_afterMedian)))
ax6.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img3,imgZI_afterAritmetic))+' si MAE: ' + str(MAE(img3,imgZI_afterAritmetic)))
ax1.imshow(Lista_imgZgomotGaussian[index])
ax2.imshow(imgZG_afterMedian)
ax3.imshow(imgZG_afterAritmetic)
ax4.imshow(Lista_imgZgomotImpulsiv[index])
ax5.imshow(imgZI_afterMedian)
ax6.imshow(imgZI_afterAritmetic)
plt.show()

### Imaginea 4

In [ ]:
index=3
fig=plt.figure(figsize=figs)
imgZG_afterMedian=medianFilter(Lista_imgZgomotGaussian[index])
imgZG_afterAritmetic=aritmeticFilter(Lista_imgZgomotGaussian[index])


imgZI_afterMedian=medianFilter(Lista_imgZgomotImpulsiv[index])
imgZI_afterAritmetic=aritmeticFilter(Lista_imgZgomotImpulsiv[index])



fig.suptitle('Pentru imaginea 4:', fontsize=50)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea afectata de zgomot gaussian')
ax2.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul median PSNR: '+ str(PSNR(img4,imgZG_afterMedian))+' si MAE: ' + str(MAE(img4,imgZG_afterMedian)))
ax3.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img4,imgZG_afterAritmetic))+' si MAE: ' + str(MAE(img4,imgZG_afterAritmetic)))
ax4.title.set_text('Imaginea afectata de zgomot impulsiv')
ax5.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul median PSNR: '+ str(PSNR(img4,imgZI_afterMedian))+' si MAE: ' + str(MAE(img4,imgZI_afterMedian)))
ax6.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img4,imgZI_afterAritmetic))+' si MAE: ' + str(MAE(img4,imgZI_afterAritmetic)))
ax1.imshow(Lista_imgZgomotGaussian[index])
ax2.imshow(imgZG_afterMedian)
ax3.imshow(imgZG_afterAritmetic)
ax4.imshow(Lista_imgZgomotImpulsiv[index])
ax5.imshow(imgZI_afterMedian)
ax6.imshow(imgZI_afterAritmetic)
plt.show()

### Imaginea 5

In [ ]:
index=4
fig=plt.figure(figsize=figs)
imgZG_afterMedian=medianFilter(Lista_imgZgomotGaussian[index])
imgZG_afterAritmetic=aritmeticFilter(Lista_imgZgomotGaussian[index])


imgZI_afterMedian=medianFilter(Lista_imgZgomotImpulsiv[index])
imgZI_afterAritmetic=aritmeticFilter(Lista_imgZgomotImpulsiv[index])



fig.suptitle('Pentru imaginea 5:', fontsize=50)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea afectata de zgomot gaussian')
ax2.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul median PSNR: '+ str(PSNR(img5,imgZG_afterMedian))+' si MAE: ' + str(MAE(img5,imgZG_afterMedian)))
ax3.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img5,imgZG_afterAritmetic))+' si MAE: ' + str(MAE(img5,imgZG_afterAritmetic)))
ax4.title.set_text('Imaginea afectata de zgomot impulsiv')
ax5.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul median PSNR: '+ str(PSNR(img5,imgZI_afterMedian))+' si MAE: ' + str(MAE(img5,imgZI_afterMedian)))
ax6.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img5,imgZI_afterAritmetic))+' si MAE: ' + str(MAE(img5,imgZI_afterAritmetic)))
ax1.imshow(Lista_imgZgomotGaussian[index])
ax2.imshow(imgZG_afterMedian)
ax3.imshow(imgZG_afterAritmetic)
ax4.imshow(Lista_imgZgomotImpulsiv[index])
ax5.imshow(imgZI_afterMedian)
ax6.imshow(imgZI_afterAritmetic)
plt.show()

### Imaginea 6

In [ ]:
index=5
fig=plt.figure(figsize=figs)
imgZG_afterMedian=medianFilter(Lista_imgZgomotGaussian[index])
imgZG_afterAritmetic=aritmeticFilter(Lista_imgZgomotGaussian[index])


imgZI_afterMedian=medianFilter(Lista_imgZgomotImpulsiv[index])
imgZI_afterAritmetic=aritmeticFilter(Lista_imgZgomotImpulsiv[index])



fig.suptitle('Pentru imaginea 6:', fontsize=50)
ax1 = fig.add_subplot(611)
ax2 = fig.add_subplot(612)
ax3 = fig.add_subplot(613)
ax4 = fig.add_subplot(614)
ax5 = fig.add_subplot(615)
ax6 = fig.add_subplot(616)
ax1.title.set_text('Imaginea afectata de zgomot gaussian')
ax2.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul median PSNR: '+ str(PSNR(img6,imgZG_afterMedian))+' si MAE: ' + str(MAE(img6,imgZG_afterMedian)))
ax3.title.set_text('Imaginea afectata de zgomot gaussian dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img6,imgZG_afterAritmetic))+' si MAE: ' + str(MAE(img6,imgZG_afterAritmetic)))
ax4.title.set_text('Imaginea afectata de zgomot impulsiv')
ax5.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul median PSNR: '+ str(PSNR(img6,imgZI_afterMedian))+' si MAE: ' + str(MAE(img6,imgZI_afterMedian)))
ax6.title.set_text('Imaginea afectata de zgomot impulsiv dupa filtrul medie aritmetica PSNR: '+ str(PSNR(img6,imgZI_afterAritmetic))+' si MAE: ' + str(MAE(img6,imgZI_afterAritmetic)))
ax1.imshow(Lista_imgZgomotGaussian[index])
ax2.imshow(imgZG_afterMedian)
ax3.imshow(imgZG_afterAritmetic)
ax4.imshow(Lista_imgZgomotImpulsiv[index])
ax5.imshow(imgZI_afterMedian)
ax6.imshow(imgZI_afterAritmetic)
plt.show()

In [ ]:
def dist(x,y):
    return np.sqrt((x[0]-y[0])**2 + (x[1]-y[1])**2 +(x[2]-y[2])**2)

import cv2
import numpy as np
img =  cv2.imread('img/pic1.jpg')

A = img [100:103,100:103,:]
#A[1,1,1] = 255
print(A)
A = A.astype(np.float32)/255
print(A)
D = np.zeros([3,3])

for i in range(3):
    for j in range(3):
        
        for k in range(3):
            for l in range(3):
                if k==1 and l == 1:
                    continue
                D[i,j] += dist(A[i,j,:],A[k,l,:])
 
print(D)
#print(np.sort(D,axis=None))

